In [16]:
import pandas as pd
import dask.dataframe as dd
from dask.delayed import delayed
from zipfile import ZipFile

import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
def load_zipped_csv(file_path):
    zipped_file = [delayed(pd.read_csv)(file_path)]
    return dd.from_delayed(zipped_file)

## Load the data

In [18]:
df_order_products__prior = load_zipped_csv("../data/order_products__prior.csv.zip")
df_orders = load_zipped_csv("../data/orders.csv.zip")
df_products = load_zipped_csv("../data/products.csv.zip")

df_aisles = load_zipped_csv("../data/aisles.csv.zip")
df_departments = load_zipped_csv("../data/departments.csv.zip")

# df_orders = df_orders.filter(pl.col("eval_set") == "prior")

In [19]:
# df = df_orders.join(df_order_products__prior, on='order_id')
# df = df.join(df_products, on='product_id')
# df = df.join(df_aisles, on='aisle_id')
# df = df.join(df_departments, on='department_id')

# df.head(5)

## Straight forward data visualization

In [20]:
# fig, ax = plt.subplots(figsize=(10, 6))
# ax.set_title("When do customer order products?")
# ax = sns.histplot(data=df, x="order_hour_of_day", bins=12, stat="probability")
# plt.show()

In [21]:
# fig, ax = plt.subplots(figsize=(10, 6))
# ax.set_title("When do customer order products?")
# ax = sns.histplot(data=df, x="order_hour_of_day", bins=12, stat="probability")
# plt.show()